# 3. 협업 필터링 추천 시스템
* 실제 취향을 고려한 개인화 추천 시스템
* 이러한 아이디어로 만들어진 추천 알고리즘이 협업 필터링,   
  콜라보레이티브 필터링 CF
* 어떤 아이템에 대해 비슷한 취향을 가진 사람들은,  
  다른 아이템 또한 비슷한 취향을 가질 것이다

## 3.1. 협업 필터링의 원리
* 추천 대상과 취향이 비슷한 사람들의 집단이 먼저 만들어져 있다
* 취향이 비슷한 사람들을 찾아내기만 하면,  
  그 사람들이 좋아하는 아이템을 추천하면 된다  
* 유사도 계산 필요

## 3.2. 유사도 지표
* CF에서 사용자 간 유사도를 구하는 것이 핵심

### 3.2.1. 상관계수
* 가장 이해하기 쉬운 유사도
* -1과 +1 사이의 값  
  * -1이면 부적 상관 관계, 음의 상관 관계 (우하향?)
  * +1이면 양의 상관 관계 (우상향?)
  * 0이면 지표가 점으로 나타나 연속적이지 않음, 유사하지 않게 나타남, 유사도가 0이라는 뜻  
  * 그런데 대문자 U 모양으로 쿼드러틱하게 나타나는 경우도 유사도가 0으로 표현된다
  * 단순히 상관 관계가 0이라고 해서 유사도가 없다고 생각하면 안 되고,  
    데이터를 실제로 비주얼라이제이션 해보거나 해서 정확한 인사이트를 얻어야 한다
* 평가 자료가 연속형인 경우 가장 이해하기 쉬운 유사도

### 3.2.2. 코사인 유사도
* 협업 필터링에서 가장 널리 쓰이는 유사도
* 각 아이템 → 하나의 차원, 사용자의 평가값 → 좌표값
  * 예를 들면 x축은 아이템A, y축은 아이템B를 나타내고  
    점 C(x1, y1)에서 x1과 y1은 사용자C가 아이템A와 아이템B 각각에 대해 평가한 평가값을 나타낸다
* 두 사용자의 평가값 유사하다  
  → 코사인theta에서 theta는 작아지고 코사인 값은 커짐  
  (theta : 점A와 점B를 각각 원점과 이은 직선 사이의 각)
* -1과 +1 사이의 값
* 데이터가 이진값이라면(샀다 안 샀다/봤다 안 봤다) 타니모토 계수 사용 권장  

### 3.2.3. 자카드 계수
* 타니모토 계수의 변형
* 이진수 데이터라면 협업 필터링에서 좋은 결과 나타난다

## 3.3. 기본 CF 알고리즘
> 1. 모든 사용자 간 평가의 유사도 계산  
> 2. 추천 대상과 다른 사용자 간 유사도 추출
> 3. 추천 대상이 평가하지 않은 아이템에 대한 예상 평가값 계산  
  → 평가값 = 다른 사용자 평가 × 다른 사용자 유사도
> 4. 아이템 중에서 예상 평가값 가장 높은 N개 추천  



In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
# 데이터 불러오기
base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

## user 데이터
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")
users = users.set_index("user_id")

## movie 데이터
u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
items = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")
items = items.set_index("movie_id")

## ratings 데이터
u_ratings_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_ratings_src, sep = "\t", names = r_cols, encoding = "latin-1")

In [10]:
# 필요한 함수 정의

## 정확도 지표 RMSE - 낮을수록 정확
def RMSE(y_true, y_pred) :
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

## score
def score(model) :
  # 테스트 데이터의 user_id와 movie_id의 쌍을 맞춰 튜플형 원소 리스트 데이터 생성
  id_pairs = zip(x_test["user_id"], x_test["movie_id"])

  # 모든 사용자-영화 쌍에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])

  # 실제 평점값
  y_true = np.array(x_test["rating"])

  return RMSE(y_true, y_pred)

In [19]:
# 데이터 셋 생성

x = ratings.copy()
y = ratings["user_id"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

ratings_matrix = x_train.pivot(index = "user_id", columns = "movie_id", values = "rating")

In [20]:
print(x_train)
print(y_train)
print(ratings_matrix)

       user_id  movie_id  rating  timestamp
20083       90       151       2  891385190
78556      347       148       3  881652888
19132       43       879       4  876159838
17096      329       250       3  891656639
48036      429       928       2  882386849
...        ...       ...     ...        ...
20187      346       132       4  875261235
72946      684       217       2  875811965
37908      561       202       3  885808867
82895      125      1037       2  892839143
16369       13       377       1  882399219

[75000 rows x 4 columns]
20083     90
78556    347
19132     43
17096    329
48036    429
        ... 
20187    346
72946    684
37908    561
82895    125
16369     13
Name: user_id, Length: 75000, dtype: int64
movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          NaN   3.0   4.0   3.0   3.0   5.0   4.0   1.0   NaN   3.0  ...   
2          NaN   NaN   NaN   

In [25]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

## 일단 코사인 유사도를 계산하기 위해
## ratings_matrix 복사하고 NaN값 0으로 대체
matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)

user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.121875,0.019116,0.049772,0.286653,0.312590,0.321495,0.232803,0.058062,0.279873,...,0.243606,0.070810,0.205655,0.142205,0.150244,0.070972,0.242357,0.079187,0.132284,0.271730
2,0.121875,1.000000,0.116889,0.142336,0.037715,0.202656,0.105917,0.061447,0.067661,0.121975,...,0.090549,0.145555,0.223168,0.295365,0.232919,0.109563,0.148461,0.103090,0.138663,0.083169
3,0.019116,0.116889,1.000000,0.228683,0.026798,0.033430,0.046410,0.043342,0.000000,0.032269,...,0.032046,0.055326,0.098206,0.044598,0.087019,0.018704,0.128555,0.080731,0.131058,0.000000
4,0.049772,0.142336,0.228683,1.000000,0.000000,0.043111,0.072188,0.164630,0.000000,0.063462,...,0.043688,0.047565,0.084430,0.148856,0.151287,0.038593,0.150203,0.178474,0.076152,0.060155
5,0.286653,0.037715,0.026798,0.000000,1.000000,0.162825,0.258649,0.221122,0.038155,0.152344,...,0.295881,0.069897,0.081709,0.055820,0.096007,0.032834,0.170548,0.093558,0.123404,0.211947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.070972,0.109563,0.018704,0.038593,0.032834,0.090762,0.110808,0.073144,0.000000,0.067393,...,0.050991,0.402265,0.205681,0.118829,0.262055,1.000000,0.046004,0.068511,0.039151,0.105537
940,0.242357,0.148461,0.128555,0.150203,0.170548,0.240248,0.196687,0.140521,0.028227,0.219506,...,0.234347,0.087884,0.130837,0.087108,0.126832,0.046004,1.000000,0.147151,0.184330,0.157605
941,0.079187,0.103090,0.080731,0.178474,0.093558,0.136322,0.054758,0.027061,0.134649,0.088880,...,0.039302,0.187347,0.283911,0.168161,0.221347,0.068511,0.147151,1.000000,0.068973,0.072304


In [28]:
# 주어진 movie_id 영화의 가중평균 rating 계산하는 함수
def CF_simple(user_id, movie_id) :
  if movie_id in ratings_matrix.columns :
    # 해당 사용자와 다른 사용자 간의 유사성만 추출
    similarity_scores = user_similarity[user_id].copy()

    # 주어진 영화에 대한 다른 사용자들의 평가
    movie_ratings = ratings_matrix[movie_id].copy()

    # 주어진 영화에 대해 평가를 하지 않은 사용자들 가중평균에서 제외하기 위해
    # 해당 사용자들의 index 추출하고
    # 평가하지 않은 애들 제외시키기
    non_rating_idx = movie_ratings[movie_ratings.isnull()].index
    similarity_scores = similarity_scores.drop(non_rating_idx)
    movie_ratings = movie_ratings.dropna()

    # 주어진 영화에 대해 평가한 사용자들의 평점을 유사도로 가중평균한 유사도 similarity_scores
    # 주어진 영화에 대한 평점 movie_ratings를 가중평균하고
    # 전체 유사도에서 sum을 나누어주면 mean_rating을 얻을 수 있다
    mean_rating = np.dot(similarity_scores, movie_ratings) / similarity_scores.sum()

  # movie_id가 없으면
  else :
    mean_rating = 3.0

  return mean_rating

In [29]:
# 정확도 계산
score(CF_simple)
## 약 1.010051245392655

1.010051245392655

## 3.4. 이웃을 고려한 CF